In [1]:
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd
import numpy as np
import time
import requests
import pymongo
from pymongo import MongoClient
import datetime

In [2]:
permit_csv = 'permitsbyus.csv'

In [3]:
# Read US Census CSV file
us_detail = pd.read_csv(permit_csv)
#us_detail.dtypes

In [15]:
# Convert US data to dataframe
us_df = us_detail.filter(['Year','1'])
us_df['1'] = round(us_df['1']).astype(int)
us_df.rename(columns={'1': 'US'}, inplace=True)
#us_df

,Year,US
0,1986,73696
1,1987,78198
2,1988,82815
3,1989,83002
4,1990,72310
5,1991,69723
6,1992,87072
7,1993,97119
8,1994,109294
9,1995,104739


In [16]:
 executable_path = {'executable_path': 'chromedriver.exe'}
 browser = Browser('chrome', **executable_path, headless=False)

In [17]:
# Texas permit data
texas_url = 'https://www.recenter.tamu.edu/data/building-permits#!/state/Texas'
browser.visit(texas_url)
html = browser.html
soup = BeautifulSoup(html,'html.parser')

In [18]:
# Extract Texas Year and Unit Data
tx_year = []
for item in soup.findAll("tr", {"class": "ng-scope"}):
    for td in item.findAll("td")[0]:
        tx_year.append(td)
tx_units = []
for item in soup.findAll("tr", {"class": "ng-scope"}):
    for td in item.findAll("td")[1]:
        tx_units.append(td)

In [19]:
year_df = pd.DataFrame(tx_year).head(42).reset_index(drop=True)
units_df = pd.DataFrame(tx_units).head(42).reset_index(drop=True)

In [20]:
tx_df = pd.concat([year_df, units_df], axis = 1)
tx_df.columns = ['Year', 'Texas']
tx_df['Texas'] = tx_df['Texas'].str.replace(',','')
tx_df = tx_df.astype({'Year':'int','Texas':'int'})
#tx_df.dtypes
#tx_df

,Year,Texas
0,1980,67870
1,1981,66161
2,1982,78714
3,1983,103252
4,1984,84565
5,1985,67964
6,1986,59143
7,1987,43975
8,1988,35908
9,1989,36658


In [21]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Declare the database
db = client.permits_db

# Declare the collections
us_col = db.us_col
tx_col = db.tx_col

In [22]:
us_df.reset_index(inplace=True)
us_data_dict = us_df.to_dict("records")
tx_df.reset_index(inplace=True)
tx_data_dict = tx_df.to_dict("records")

In [23]:
us_col.insert_many(us_data_dict)
tx_col.insert_many(us_data_dict)